In [36]:
from pathlib import Path
import json

# try to locate the file in the repo (works for relative or absolute layouts)
candidates = list(Path.cwd().rglob("response_v4_7.json"))
if candidates:
    path = candidates[0]
else:
    path = Path("Causal_extractor/data_extract/output/response_v4_7.json")
if not path.exists():
    raise FileNotFoundError(f"response_v3.json not found. Checked: {path}")

with open(path, "r", encoding="utf-8") as f:
    data = json.load(f)

if not isinstance(data, list):
    raise ValueError("Expected JSON file to contain a list of records")

n = len(data)
keys = set().union(*(d.keys() for d in data))

na_indices = {k: [] for k in keys}
for i, rec in enumerate(data):
    for k in keys:
        v = rec.get(k, "")
        if isinstance(v, str) and v.strip() == "N/A":
            na_indices[k].append(i)

# print concise summary
for k in sorted(keys):
    cnt = len(na_indices[k])
    pct = 100 * cnt / n if n else 0
    print(f"{k}: {cnt}/{n} ({pct:.1f}%)  indices={na_indices[k]}")

explicit_type: 0/10 (0.0%)  indices=[]
marked_type: 1/10 (10.0%)  indices=[9]
marker: 0/10 (0.0%)  indices=[]
object: 0/10 (0.0%)  indices=[]
pattern_type: 0/10 (0.0%)  indices=[]
reasoning: 0/10 (0.0%)  indices=[]
relationship: 0/10 (0.0%)  indices=[]
sentence_type: 0/10 (0.0%)  indices=[]
source_text: 0/10 (0.0%)  indices=[]
subject: 0/10 (0.0%)  indices=[]


In [37]:
import pandas as pd
from IPython.display import display

# Convert loaded JSON list-of-dicts into a DataFrame and display as a table
df = pd.DataFrame(data)
print(f"Loaded {len(df)} rows, {len(df.columns)} columns")
# show a preview (adjust or remove .head to show more)
df

# To persist or inspect further:
# df.to_csv('response_v3_table_preview.csv', index=False)
# display(df)  # uncomment to render full table in the notebook

Loaded 10 rows, 10 columns


,pattern_type,sentence_type,marked_type,explicit_type,relationship,marker,subject,object,source_text,reasoning
0,C,SB,M,E,some event -> causes an unusual amount of trash,causes,some event,an unusual amount of trash,We know that university staff will call for tr...,The marker 'causes' explicitly links the event...
1,C,SB,U,E,an unusual amount of trash -> university staff...,None,an unusual amount of trash,university staff call for trash trucks,We know that university staff will call for tr...,The state of having an unusual amount of trash...
2,C,ES,M,E,limited space in our university -> garbage buf...,due to,limited space in our university,the garbage buffer is created,"Also due to limited space in our university, t...",The marker 'due to' explicitly states that the...
3,C,SB,U,E,garbage buffer -> prevents normal collection p...,None,the garbage buffer,preventing normal collection points from overf...,the garbage buffer is created to store trash p...,The purpose of the buffer is explicitly stated...
4,C,ES,M,E,creation without proper design -> makes it clo...,making,it is created without proper design,it close to human traffic,it is created without proper design making it ...,The participle 'making' acts as a causal marke...
5,C,ES,U,E,universities accept more students -> these cha...,None,universities accept more students,these challenges emerge,And these challenges emerge as universities ac...,The text explicitly states that the challenges...
6,C,ES,U,E,Increase in the number of students -> increase...,None,Increase in the number of students,increases the number of trash producers,Increase in the number of students also increa...,The verb 'increases' directly states the causa...
7,C,SB,U,E,economic feasibility -> makes the project in g...,None,economic feasibility,makes the project in general fail,He mentions that the most important factor tha...,The verb 'makes' explicitly defines economic f...
8,C,SB,U,E,Too much trash -> Call BMA for a special pickup,None,too much trash,call BMA for a special pickup,ถ้าขยะเยอะเกิน -> โทรเรียกกทมมาเก็บตาม case,The arrow '->' and context explicitly state th...
9,F,ES,N/A,E,policy -> changes according to the executive b...,None,policy,changes according to the executive board,นโยบายเปลี่ยนตามผู้บริหาร,This is a statement of fact describing how pol...


In [38]:
from collections import defaultdict
import pandas as pd
from IPython.display import display

def show_grouped_rows_table(data, key, max_rows_per_group=None):
    groups = defaultdict(list)
    for rec in data:
        groups[rec.get(key, "")].append(rec)
    for val in sorted(groups.keys(), key=lambda x: str(x)):
        rows = groups[val]
        print(f'\n--- {key!r} = {val!r} ({len(rows)} rows) ---')
        if not rows:
            print("(no rows)")
            continue
        df = pd.DataFrame(rows)
        if max_rows_per_group is not None:
            display(df.head(max_rows_per_group))
        else:
            display(df)

# show rows grouped by explicit_type, marked_type, and object (rendered as tables)
show_grouped_rows_table(data, "explicit_type", max_rows_per_group=50)


--- 'explicit_type' = 'E' (10 rows) ---


,pattern_type,sentence_type,marked_type,explicit_type,relationship,marker,subject,object,source_text,reasoning
0,C,SB,M,E,some event -> causes an unusual amount of trash,causes,some event,an unusual amount of trash,We know that university staff will call for tr...,The marker 'causes' explicitly links the event...
1,C,SB,U,E,an unusual amount of trash -> university staff...,None,an unusual amount of trash,university staff call for trash trucks,We know that university staff will call for tr...,The state of having an unusual amount of trash...
2,C,ES,M,E,limited space in our university -> garbage buf...,due to,limited space in our university,the garbage buffer is created,"Also due to limited space in our university, t...",The marker 'due to' explicitly states that the...
3,C,SB,U,E,garbage buffer -> prevents normal collection p...,None,the garbage buffer,preventing normal collection points from overf...,the garbage buffer is created to store trash p...,The purpose of the buffer is explicitly stated...
4,C,ES,M,E,creation without proper design -> makes it clo...,making,it is created without proper design,it close to human traffic,it is created without proper design making it ...,The participle 'making' acts as a causal marke...
5,C,ES,U,E,universities accept more students -> these cha...,None,universities accept more students,these challenges emerge,And these challenges emerge as universities ac...,The text explicitly states that the challenges...
6,C,ES,U,E,Increase in the number of students -> increase...,None,Increase in the number of students,increases the number of trash producers,Increase in the number of students also increa...,The verb 'increases' directly states the causa...
7,C,SB,U,E,economic feasibility -> makes the project in g...,None,economic feasibility,makes the project in general fail,He mentions that the most important factor tha...,The verb 'makes' explicitly defines economic f...
8,C,SB,U,E,Too much trash -> Call BMA for a special pickup,None,too much trash,call BMA for a special pickup,ถ้าขยะเยอะเกิน -> โทรเรียกกทมมาเก็บตาม case,The arrow '->' and context explicitly state th...
9,F,ES,N/A,E,policy -> changes according to the executive b...,None,policy,changes according to the executive board,นโยบายเปลี่ยนตามผู้บริหาร,This is a statement of fact describing how pol...


In [39]:
show_grouped_rows_table(data, "marked_type", max_rows_per_group=50)



--- 'marked_type' = 'M' (3 rows) ---


,pattern_type,sentence_type,marked_type,explicit_type,relationship,marker,subject,object,source_text,reasoning
0,C,SB,M,E,some event -> causes an unusual amount of trash,causes,some event,an unusual amount of trash,We know that university staff will call for tr...,The marker 'causes' explicitly links the event...
1,C,ES,M,E,limited space in our university -> garbage buf...,due to,limited space in our university,the garbage buffer is created,"Also due to limited space in our university, t...",The marker 'due to' explicitly states that the...
2,C,ES,M,E,creation without proper design -> makes it clo...,making,it is created without proper design,it close to human traffic,it is created without proper design making it ...,The participle 'making' acts as a causal marke...



--- 'marked_type' = 'N/A' (1 rows) ---


,pattern_type,sentence_type,marked_type,explicit_type,relationship,marker,subject,object,source_text,reasoning
0,F,ES,N/A,E,policy -> changes according to the executive b...,None,policy,changes according to the executive board,นโยบายเปลี่ยนตามผู้บริหาร,This is a statement of fact describing how pol...



--- 'marked_type' = 'U' (6 rows) ---


,pattern_type,sentence_type,marked_type,explicit_type,relationship,marker,subject,object,source_text,reasoning
0,C,SB,U,E,an unusual amount of trash -> university staff...,None,an unusual amount of trash,university staff call for trash trucks,We know that university staff will call for tr...,The state of having an unusual amount of trash...
1,C,SB,U,E,garbage buffer -> prevents normal collection p...,None,the garbage buffer,preventing normal collection points from overf...,the garbage buffer is created to store trash p...,The purpose of the buffer is explicitly stated...
2,C,ES,U,E,universities accept more students -> these cha...,None,universities accept more students,these challenges emerge,And these challenges emerge as universities ac...,The text explicitly states that the challenges...
3,C,ES,U,E,Increase in the number of students -> increase...,None,Increase in the number of students,increases the number of trash producers,Increase in the number of students also increa...,The verb 'increases' directly states the causa...
4,C,SB,U,E,economic feasibility -> makes the project in g...,None,economic feasibility,makes the project in general fail,He mentions that the most important factor tha...,The verb 'makes' explicitly defines economic f...
5,C,SB,U,E,Too much trash -> Call BMA for a special pickup,None,too much trash,call BMA for a special pickup,ถ้าขยะเยอะเกิน -> โทรเรียกกทมมาเก็บตาม case,The arrow '->' and context explicitly state th...


In [40]:
show_grouped_rows_table(data, "object", max_rows_per_group=50)


--- 'object' = 'an unusual amount of trash' (1 rows) ---


,pattern_type,sentence_type,marked_type,explicit_type,relationship,marker,subject,object,source_text,reasoning
0,C,SB,M,E,some event -> causes an unusual amount of trash,causes,some event,an unusual amount of trash,We know that university staff will call for tr...,The marker 'causes' explicitly links the event...



--- 'object' = 'call BMA for a special pickup' (1 rows) ---


,pattern_type,sentence_type,marked_type,explicit_type,relationship,marker,subject,object,source_text,reasoning
0,C,SB,U,E,Too much trash -> Call BMA for a special pickup,None,too much trash,call BMA for a special pickup,ถ้าขยะเยอะเกิน -> โทรเรียกกทมมาเก็บตาม case,The arrow '->' and context explicitly state th...



--- 'object' = 'changes according to the executive board' (1 rows) ---


...

,pattern_type,sentence_type,marked_type,explicit_type,relationship,marker,subject,object,source_text,reasoning
0,C,ES,M,E,limited space in our university -> garbage buf...,due to,limited space in our university,the garbage buffer is created,"Also due to limited space in our university, t...",The marker 'due to' explicitly states that the...



--- 'object' = 'these challenges emerge' (1 rows) ---


,pattern_type,sentence_type,marked_type,explicit_type,relationship,marker,subject,object,source_text,reasoning
0,C,ES,U,E,universities accept more students -> these cha...,None,universities accept more students,these challenges emerge,And these challenges emerge as universities ac...,The text explicitly states that the challenges...



--- 'object' = 'university staff call for trash trucks' (1 rows) ---


,pattern_type,sentence_type,marked_type,explicit_type,relationship,marker,subject,object,source_text,reasoning
0,C,SB,U,E,an unusual amount of trash -> university staff...,None,an unusual amount of trash,university staff call for trash trucks,We know that university staff will call for tr...,The state of having an unusual amount of trash...
